In [ ]:
# !pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [ ]:
import openai
import json

def topic_breakdown_agent(user_input):
    """
    This function receives a research topic from the user,
    generates subtopics or subqueries using the OpenAI API,
    and returns the extracted list of subtopics.
    """

    client = openai.OpenAI(
        api_key=openai_api_key,
    )

    system_prompt = """
    You are a research assistant. Your task is to break down any research topic into well-defined, relevant subtopics or subqueries.
    - Limit to max of 3 subtopics.
    - Your response must strictly be a JSON object with a single key "subtopics" and the value as a list of strings.
    - The subtopics should focus on covering the topic comprehensively, logically divided into meaningful areas for further research.
    - Ensure that each subtopic is phrased clearly, avoiding ambiguity.
    - Avoid including any additional text, explanations, or comments outside of the JSON object.

    ### Example Input:
    "Break down the research topic: The impact of climate change on global agriculture"

    ### Example Output:
    {
      "subtopics": [
        "Effects of rising temperatures on crop yields",
        "Impact of changing precipitation patterns on farming",
        "Role of climate change in increasing pest and disease prevalence",
        "Adaptation strategies for farmers in different regions",
        "Economic consequences of climate change on the agriculture industry"
      ]
    }

    Your role:
    - To break down any research topic provided by the user.
    - To always return a JSON object formatted as shown above, containing subtopics as a list of strings.
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Break down the research topic: {user_input}"}
        ],
        max_tokens=500,
        temperature=0,
        response_format={"type": "json_object"}
    )

    response_content = response.choices[0].message.content

    response_json = json.loads(response_content)
    return response_json.get("subtopics", [])

In [ ]:
import openai
import json

def query_expansion_agent(subtopics):
    """
    This function receives a list of subtopics,
    expands each subtopic using the OpenAI API,
    and returns the expanded queries for each subtopic.
    """

    client = openai.OpenAI(
        api_key=openai_api_key,
    )

    system_prompt = """
    You are a query expansion assistant. Your task is to expand on provided research subtopics by generating related keywords, synonyms, and detailed phrases to enhance search results.
    - For each subtopic, generate a list of keywords, synonyms, and descriptive phrases.
    - Limit to a maximum of 5 keywords/synonyms/phrases for each subtopic.
    - Your response must strictly be a JSON object with a single key "expanded_queries" and the value as a dictionary. Each key in the dictionary is a subtopic, and its value is a list of strings containing the expanded queries.
    - Avoid including any additional text, explanations, or comments outside of the JSON object.

    ### Example Input:
    "Expand on the subtopics: ['Effects of rising temperatures on crop yields', 'Impact of changing precipitation patterns on farming']"

    ### Example Output:
    {
      "expanded_queries": {
        "Effects of rising temperatures on crop yields": [
          "Temperature increase and agricultural productivity",
          "Heat stress effects on crops",
          "Impact of global warming on farming",
          "Thermal thresholds for crop growth",
          "Climate-related crop yield predictions"
        ],
        "Impact of changing precipitation patterns on farming": [
          "Drought and farming challenges",
          "Flooding effects on agricultural lands",
          "Precipitation variability and crop growth",
          "Rainfall patterns and irrigation needs",
          "Water scarcity in farming regions"
        ]
      }
    }

    Your role:
    - Receive a list of subtopics.
    - Generate related keywords, synonyms, and phrases for each subtopic.
    - Return a JSON object formatted as shown in the example.
    """

    formatted_input = f"Expand on the subtopics: {json.dumps(subtopics)}"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": formatted_input}
        ],
        max_tokens=1000,
        temperature=0,
        response_format={"type": "json_object"}
    )

    response_content = response.choices[0].message.content

    response_json = json.loads(response_content)
    return response_json.get("expanded_queries", {})

In [ ]:
from google.colab import userdata
you_api_key = userdata.get('you_api_key')

In [ ]:
import requests
import json

def get_ai_snippets_for_query(query):
    """
    Fetch AI-generated snippets for a given query using the You API.
    """
    headers = {"X-API-Key": you_api_key}
    params = {"query": query}
    response = requests.get(
        "https://api.ydc-index.io/search",
        params=params,
        headers=headers,
    )
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"API call failed with status code {response.status_code}"}

def fetch_results_for_subtopics(subtopics_list, max_results=3):
    """
    Fetch search results for a list of subtopics using the You API.
    Limit the output to a maximum number of results for each query.
    """
    results = {}
    for subtopic in subtopics_list:
        print(f"Fetching results for subtopic: {subtopic}")
        api_result = get_ai_snippets_for_query(subtopic)

        if "hits" in api_result:

            snippets = [
                {
                    "title": hit.get("title", "No Title"),
                    "description": hit.get("description", "No Description"),
                    "url": hit.get("url", "No URL"),
                }
                for hit in api_result["hits"][:max_results]
            ]
            results[subtopic] = snippets
        else:
            results[subtopic] = [{"error": "No results found."}]
    return results

In [ ]:
import openai
import json

def research_synthesis_agent(expanded_queries, search_results):
    """
    Combines expanded queries and search results to generate detailed enhanced responses for each subtopic.
    """

    client = openai.OpenAI(
        api_key=openai_api_key,
    )

    system_prompt = """
    You are a research synthesis assistant. Your task is to combine query expansion results with search results for a set of subtopics to generate a detailed and enhanced response for each subtopic.
    - For each subtopic, you must merge the expanded queries and search snippets into a cohesive and enhanced response.
    - Each response should provide a detailed explanation, integrating key points from both expanded queries and search results.
    - Structure the response into logical paragraphs where necessary, and ensure it covers all key aspects of the topic.
    - Each response must be at least three sentences long, ensuring that all relevant information is included. If possible, elaborate on implications, challenges, and future directions related to the topic.
    - Your response must strictly be a JSON object with a single key "enhanced_responses" and the value as a dictionary. Each key in the dictionary is a subtopic, and its value is a single string containing the enhanced response.
    - Avoid including any additional text, explanations, or comments outside of the JSON object.

    ### Example Input:
    {
      "expanded_queries": {
        "Effects of rising temperatures on crop yields": [
          "Temperature increase and agricultural productivity",
          "Heat stress effects on crops",
          "Impact of global warming on farming"
        ]
      },
      "search_results": {
        "Effects of rising temperatures on crop yields": [
          "Rising temperatures could reduce crop yields by 30%.",
          "Heat waves affect soil moisture and growth rates."
        ]
      }
    }

    ### Example Output:
    {
      "enhanced_responses": {
        "Effects of rising temperatures on crop yields": "Rising temperatures pose significant challenges to agricultural productivity, with studies showing up to a 30% reduction in crop yields. Key concerns include heat stress effects on crops, soil moisture loss during heat waves, and changing growth rates due to global warming. Moreover, the combination of higher temperatures and altered precipitation patterns creates additional stress on farming systems, particularly in tropical and subtropical regions. To address these challenges, researchers are exploring adaptive strategies such as the development of heat-resistant crop varieties, improved irrigation systems, and integrated pest management approaches."
      }
    }

    Your role:
    - Combine query expansion and search results for each subtopic.
    - Generate a detailed, multi-paragraph response that incorporates all relevant information, providing at least three sentences for each subtopic.
    - Return a JSON object formatted as shown in the example.
    """

    input_data = {
        "expanded_queries": expanded_queries,
        "search_results": search_results
    }

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": json.dumps(input_data)}
        ],
        max_tokens=3000,
        temperature=0,
        response_format={"type": "json_object"}
    )

    response_content = response.choices[0].message.content

    response_json = json.loads(response_content)
    return response_json.get("enhanced_responses", {})

In [ ]:
import openai
import json

def summarizer_agent(enhanced_responses):
    """
    Summarizes enhanced responses into detailed 7-8 sentence summaries for each subtopic.
    """

    client = openai.OpenAI(
        api_key=openai_api_key,
    )

    summarizer_prompt = """
    You are a detailed summarization assistant. Your task is to summarize the enhanced responses from a previous agent into comprehensive summaries for each subtopic.
    - Each summary must be atleast 2 paragraphs long and should provide an in-depth synthesis of the content.
    - Identify the key points, implications, challenges, and possible solutions mentioned in the enhanced responses.
    - Ensure the summaries are coherent, detailed, and provide a complete understanding of the subtopic without leaving out critical information.
    - Use formal language, logical structuring, and well-defined paragraphs to improve readability.
    - Avoid simply rephrasing the enhanced responses verbatim; instead, synthesize the information into a clearer and more holistic summary.
    - Your output must be a JSON object with a single key "summaries" and the value as a dictionary. Each key in the dictionary corresponds to a subtopic, and its value is a string containing the detailed summary.
    - Do not include any additional text, explanations, or comments outside of the JSON object.

    ### Example Input:
    {
      "enhanced_responses": {
        "Effects of rising temperatures on crop yields": "Rising temperatures pose significant challenges to agricultural productivity, with studies showing up to a 30% reduction in crop yields. Key concerns include heat stress effects on crops, soil moisture loss during heat waves, and changing growth rates due to global warming. Moreover, the combination of higher temperatures and altered precipitation patterns creates additional stress on farming systems, particularly in tropical and subtropical regions. To address these challenges, researchers are exploring adaptive strategies such as the development of heat-resistant crop varieties, improved irrigation systems, and integrated pest management approaches."
      }
    }

    ### Example Output:
    {
      "summaries": {
        "Effects of rising temperatures on crop yields": "The increase in global temperatures significantly threatens agricultural productivity, potentially reducing crop yields by as much as 30%. This is largely attributed to heat stress, which hinders crop growth, combined with the loss of soil moisture during prolonged heat waves. Global warming also alters growth rates and disrupts the balance of farming ecosystems. Particularly vulnerable are regions in tropical and subtropical climates, where precipitation patterns are becoming increasingly erratic. Addressing these challenges requires a multifaceted approach. Key solutions include developing heat-resistant crop varieties to withstand extreme temperatures, enhancing irrigation systems to optimize water use, and implementing integrated pest management to combat new pest challenges arising from changing weather patterns. Researchers are also investigating ways to increase soil resilience and improve farming practices to adapt to these environmental changes. A concerted effort across scientific, agricultural, and policy sectors is essential to mitigate the long-term impacts of rising temperatures on global food security."
      }
    }
    """

    input_data = {
        "enhanced_responses": enhanced_responses
    }

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": summarizer_prompt},
            {"role": "user", "content": json.dumps(input_data)}
        ],
        max_tokens=3000,
        temperature=0,
        response_format={"type": "json_object"}
    )

    response_content = response.choices[0].message.content

    response_json = json.loads(response_content)
    return response_json.get("summaries", {})

In [ ]:
import openai
import json

def critique_tool(summaries):
    """
    Critiques summaries, offers improvement suggestions, and proposes related topics.
    """

    client = openai.OpenAI(
        api_key=openai_api_key,
    )

    critique_tool_prompt = """
    You are a critical evaluation assistant. Your task is to review the summaries provided for a given set of subtopics, identify areas for improvement, and suggest enhancements to make the summaries more comprehensive and insightful. Additionally, propose related topics that might enrich understanding or provide further context.
    - For each summary, provide a detailed critique of its content, structure, and completeness.
    - Highlight strengths (e.g., clarity, relevance) and weaknesses (e.g., missing details, lack of depth, logical gaps).
    - Offer specific suggestions to improve the summary, such as elaborating on key points, addressing omitted information, or rephrasing for better readability.
    - Suggest 2-3 additional related topics that could complement or enhance the original subtopic.
    - The response must be in detail with atleast 2 paragraphs.
    - Structure your response in a logical and detailed manner:
      1. Start with a critique (strengths and weaknesses).
      2. Follow with specific improvement suggestions.
      3. Conclude with related topics for further exploration.
    - Your output must strictly be a JSON object with a single key "critiques" and the value as a dictionary. Each key in the dictionary is a subtopic, and its value is an object with three keys: "critique", "improvement_suggestions", and "related_topics".
      - "critique" contains a string with the strengths and weaknesses of the summary.
      - "improvement_suggestions" contains a string with detailed suggestions for improvement.
      - "related_topics" contains a list of 2-3 strings, each being a related topic.

    ### Example Input:
    {
      "summaries": {
        "Effects of rising temperatures on crop yields": "The increase in global temperatures significantly threatens agricultural productivity, potentially reducing crop yields by as much as 30%. This is largely attributed to heat stress, which hinders crop growth, combined with the loss of soil moisture during prolonged heat waves. Global warming also alters growth rates and disrupts the balance of farming ecosystems. Particularly vulnerable are regions in tropical and subtropical climates, where precipitation patterns are becoming increasingly erratic. Addressing these challenges requires a multifaceted approach. Key solutions include developing heat-resistant crop varieties to withstand extreme temperatures, enhancing irrigation systems to optimize water use, and implementing integrated pest management to combat new pest challenges arising from changing weather patterns. Researchers are also investigating ways to increase soil resilience and improve farming practices to adapt to these environmental changes. A concerted effort across scientific, agricultural, and policy sectors is essential to mitigate the long-term impacts of rising temperatures on global food security."
      }
    }

    ### Example Output:
    {
      "critiques": {
        "Effects of rising temperatures on crop yields": {
          "critique": "The summary provides a solid overview of the challenges posed by rising temperatures on agriculture. It effectively highlights key threats such as heat stress, soil moisture loss, and erratic precipitation patterns. The proposed solutions, including heat-resistant crops and improved irrigation, are relevant and well-articulated. However, the summary could delve deeper into the economic implications of reduced yields and their impact on global food security. Additionally, it lacks a discussion of regional disparities, particularly the different challenges faced by developed and developing nations.",
          "improvement_suggestions": "Expand on the economic consequences of reduced agricultural productivity, particularly in terms of global food prices and farmer livelihoods. Include examples of regions or crops most affected by these changes, such as staple crops like rice and wheat. Discuss the role of international cooperation and policy in addressing these issues. Finally, rephrase certain sections to ensure smoother transitions between challenges and solutions for better readability.",
          "related_topics": [
            "The role of climate-smart agriculture in combating global warming",
            "Economic impacts of climate change on global food systems",
            "The influence of changing precipitation patterns on water resources"
          ]
        }
      }
    }
    """

    input_data = {
        "summaries": summaries
    }

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": critique_tool_prompt},
            {"role": "user", "content": json.dumps(input_data)}
        ],
        max_tokens=3000,
        temperature=0.2,
        response_format={"type": "json_object"}
    )

    response_content = response.choices[0].message.content
    response_json = json.loads(response_content)
    return response_json.get("critiques", {})

In [ ]:
import openai
import json

def final_summarized_agent(critiques):
    """
    Takes critiques and synthesizes them into final user-friendly summaries.
    """

    client = openai.OpenAI(
        api_key=openai_api_key,
    )

    final_summary_prompt = """
    You are a final summarization assistant. Your task is to take critiques and suggestions for improvement, synthesize them, and produce a clean, concise, and user-friendly summary for each subtopic.
    - Focus on clarity, coherence, and readability to ensure the summary is easily understood by a general audience.
    - Highlight the main points, including key challenges, solutions, and any notable implications or insights.
    - Integrate the critique’s suggestions for improvement to create a well-rounded, improved version of the original summary.
    - Ensure the summary is detailed but avoids unnecessary technical jargon, making it accessible for non-experts.
    - Each summary should be structured into logical paragraphs, with 7-8 sentences providing a comprehensive yet clear overview.
    - Avoid repeating the critique’s content verbatim; instead, synthesize it into a polished, final summary.
    - Your output must strictly be a JSON object with a single key "final_summaries" and the value as a dictionary. Each key in the dictionary is a subtopic, and its value is a string containing the final user-friendly summary.

    ### Example Input:
    {
      "critiques": {
        "Effects of rising temperatures on crop yields": {
          "critique": "The summary provides a solid overview of the challenges posed by rising temperatures on agriculture. It effectively highlights key threats such as heat stress, soil moisture loss, and erratic precipitation patterns. The proposed solutions, including heat-resistant crops and improved irrigation, are relevant and well-articulated. However, the summary could delve deeper into the economic implications of reduced yields and their impact on global food security. Additionally, it lacks a discussion of regional disparities, particularly the different challenges faced by developed and developing nations.",
          "improvement_suggestions": "Expand on the economic consequences of reduced agricultural productivity, particularly in terms of global food prices and farmer livelihoods. Include examples of regions or crops most affected by these changes, such as staple crops like rice and wheat. Discuss the role of international cooperation and policy in addressing these issues. Finally, rephrase certain sections to ensure smoother transitions between challenges and solutions for better readability.",
          "related_topics": [
            "The role of climate-smart agriculture in combating global warming",
            "Economic impacts of climate change on global food systems",
            "The influence of changing precipitation patterns on water resources"
          ]
        }
      }
    }

    ### Example Output:
    {
      "final_summaries": {
        "Effects of rising temperatures on crop yields": "Rising temperatures significantly threaten agricultural productivity, with potential crop yield reductions of up to 30%. Key challenges include heat stress, loss of soil moisture during prolonged heat waves, and increasingly erratic precipitation patterns, particularly in tropical and subtropical regions. These changes not only disrupt crop growth and ecosystem balance but also have serious economic implications, including rising global food prices and reduced farmer incomes. Staple crops like rice and wheat are especially vulnerable to these conditions. Addressing these issues requires a multifaceted approach, such as developing heat-resistant crop varieties, improving irrigation systems for efficient water use, and implementing integrated pest management strategies. International cooperation and policy-making are also essential to mitigate the disparities between developed and developing nations, which face unique challenges. Researchers continue to explore innovative solutions to enhance soil resilience and adapt farming practices to climate change. Together, these efforts aim to secure food systems and global food security in the face of warming temperatures."
      }
    }
    """
    input_data = {
        "critiques": critiques
    }

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": final_summary_prompt},
            {"role": "user", "content": json.dumps(input_data)}
        ],
        max_tokens=3000,
        temperature=0.5,
        response_format={"type": "json_object"}
    )

    response_content = response.choices[0].message.content

    response_json = json.loads(response_content)
    return response_json.get("final_summaries", {})

In [ ]:
if __name__ == "__main__":
    # User input
    user_input = input("Enter a research topic: ")
    print("\nResearch Topic:", user_input)

    # Agent 1
    subtopics_list = topic_breakdown_agent(user_input)
    print("\nSubtopics:")
    for idx, subtopic in enumerate(subtopics_list, start=1):
        print(f"  {idx}. {subtopic}")

    # Agent 2
    expanded_queries = query_expansion_agent(subtopics_list)
    print("\nExpanded Queries:")
    for subtopic, queries in expanded_queries.items():
        print(f"  {subtopic}:")
        for query in queries:
            print(f"    - {query}")

    # YOU API for real-time search
    search_results = fetch_results_for_subtopics(subtopics_list, max_results=3)
    print("\nSearch Results:")
    for subtopic, results in search_results.items():
        print(f"  {subtopic}:")
        for result in results:
            print(f"    - Title: {result['title']}")
            print(f"      Description: {result['description']}")
            print(f"      URL: {result['url']}")

    # Agent 3
    enhanced_responses = research_synthesis_agent(expanded_queries, search_results)
    print("\nEnhanced Responses:")
    for subtopic, response in enhanced_responses.items():
        print(f"  {subtopic}: {response}")

    # Agent 4
    summaries = summarizer_agent(enhanced_responses)
    print("\nSummaries:")
    for subtopic, summary in summaries.items():
        print(f"  {subtopic}: {summary}")

    # Agent 5
    critiques = critique_tool(summaries)
    print("\nCritiques:")
    for subtopic, critique_data in critiques.items():
        print(f"  {subtopic}:")
        for key, value in critique_data.items():
            print(f"    {key.capitalize()}: {value}")

    # Agent 6
    final_summaries = final_summarized_agent(critiques)
    print("\nFinal Summaries:")
    for subtopic, final_summary in final_summaries.items():
        print(f"  {subtopic}: {final_summary}")

Enter a research topic: self driving cars

Research Topic: self driving cars

Subtopics:
  1. Technological advancements in autonomous vehicle systems
  2. Challenges and limitations of self-driving car technology
  3. Ethical and legal considerations of autonomous vehicles

Expanded Queries:
  Technological advancements in autonomous vehicle systems:
    - Innovations in self-driving car technology
    - Advances in autonomous vehicle sensors
    - Development of driverless vehicle software
    - Autonomous vehicle navigation systems
    - Emerging technologies in self-driving cars
  Challenges and limitations of self-driving car technology:
    - Obstacles to autonomous vehicle adoption
    - Issues with self-driving car safety
    - Limitations of driverless vehicle technology
    - Challenges in autonomous vehicle decision-making
    - Barriers to self-driving car implementation
  Ethical and legal considerations of autonomous vehicles:
    - Ethical dilemmas in self-driving car pr